In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean

/data/hieunm/anaconda3/envs/action-env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_df = pd.read_csv('./n_train.csv')
test_df = pd.read_csv('./n_test.csv')

In [3]:
exclude_cols = ['index', 'startdate']
temporal_attrs = ['year', 'quarter', 'month', 'week', 'dayofyear', 'season', 'day_of_year_sin', 'day_of_year_cos', 'week_sin', 'week_cos', 'month_sin', 'month_cos', 'season_sin', 'season_cos', 'quarter_sin', 'quarter_cos']
loc_attrs = ['lat', 'lon', 'loc_group']
embedding_attrs = ['climateregions__climateregion']
target=["contest-tmp2m-14d__tmp2m"]
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in temporal_attrs and c not in loc_attrs and c not in target and c not in embedding_attrs]
print("Main features:", len(main_attrs))

Main features: 240


In [4]:
train_independent_corr = train_df[main_attrs + temporal_attrs + loc_attrs + embedding_attrs].corr()
train_seleted_corr_columns = np.full((train_independent_corr.shape[0],), True, dtype=bool)
for i in range(train_independent_corr.shape[0]):
    for j in range(i + 1, train_independent_corr.shape[0]):
        if train_independent_corr.iloc[i, j] >= 0.85:
            
            if train_seleted_corr_columns[j]:
                train_seleted_corr_columns[j] = False
train_selected_columns = train_df[main_attrs + temporal_attrs + loc_attrs + embedding_attrs].columns[train_seleted_corr_columns]

In [5]:
out_cols = ['icec-2010-1',
 'icec-2010-2',
 'icec-2010-3',
 'icec-2010-4',
 'icec-2010-5',
 'icec-2010-6',
 'icec-2010-7',
 'icec-2010-8',
 'icec-2010-9',
 'icec-2010-10',
 'year',
 'quarter',
 'month_cos',
 'quarter_sin',
 'quarter_cos']
features = [c for c in train_selected_columns if c not in out_cols]
print(features)

['contest-pevpr-sfc-gauss-14d__pevpr', 'nmme0-tmp2m-34w__cancm30', 'contest-wind-h10-14d__wind-hgt-10', 'contest-rhum-sig995-14d__rhum', 'nmme-prate-34w__cancm3', 'nmme-prate-34w__ccsm3', 'nmme-prate-34w__ccsm4', 'nmme-prate-34w__cfsv2', 'nmme-prate-34w__gfdl', 'nmme-prate-34w__gfdlflora', 'nmme-prate-34w__nasa', 'nmme0-prate-56w__cancm30', 'nmme0-prate-56w__cancm40', 'nmme0-prate-56w__ccsm30', 'nmme0-prate-56w__ccsm40', 'nmme0-prate-56w__cfsv20', 'nmme0-prate-56w__gfdlflora0', 'nmme0-prate-56w__gfdl0', 'nmme0-prate-56w__nasa0', 'nmme0-prate-34w__cancm40', 'contest-slp-14d__slp', 'contest-wind-vwnd-925-14d__wind-vwnd-925', 'contest-pres-sfc-gauss-14d__pres', 'contest-wind-uwnd-250-14d__wind-uwnd-250', 'contest-prwtr-eatm-14d__prwtr', 'contest-wind-vwnd-250-14d__wind-vwnd-250', 'contest-precip-14d__precip', 'contest-wind-h850-14d__wind-hgt-850', 'contest-wind-uwnd-925-14d__wind-uwnd-925', 'elevation__elevation', 'wind-vwnd-250-2010-1', 'wind-vwnd-250-2010-2', 'wind-vwnd-250-2010-3', 'wi

In [6]:
train = train_df[features]
test = test_df[features]

y_train = train_df[target[0]].values

In [7]:
en = make_pipeline(RobustScaler(), SelectFromModel(Lasso(alpha=0.03)), ElasticNet(alpha=0.001, l1_ratio=0.1))
    
cat = CatBoostRegressor(iterations=4900, verbose=200)

adb = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4), n_estimators=300, random_state=2017)

bag = BaggingRegressor(DecisionTreeRegressor(max_depth=4), n_estimators=300, random_state=2017, verbose=200)

rf = RandomForestRegressor(n_estimators=250, n_jobs=4, min_samples_split=25, min_samples_leaf=25, verbose=200)

et = ExtraTreesRegressor(n_estimators=250, n_jobs=4, min_samples_split=25, min_samples_leaf=25, verbose=200)

gbr = GradientBoostingRegressor(loss='huber', learning_rate=0.1, verbose=200, n_estimators=350, min_samples_split=25, min_samples_leaf=25)

xgbm = xgb.sklearn.XGBRegressor(max_depth=6, learning_rate=0.005, subsample=0.6,
                                objective='reg:linear', n_estimators=1000, verbose=200)

nn = MLPRegressor(hidden_layer_sizes=(200, 400, 50), random_state =2017, early_stopping=True, verbose=200)

svm = SVR(kernel='poly', degree=3, gamma='auto', coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=True, max_iter=-1)

In [ ]:
results = cross_val_score(bag, train.values, y_train, cv=5, scoring='neg_mean_squared_error')#0.567878 (0.058145)
print("bagging regressor score: {:4f} ({:4f})".format(results.mean(), results.std()))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Building estimator 1 of 300 for this parallel run (total 300)...
Building estimator 2 of 300 for this parallel run (total 300)...
Building estimator 3 of 300 for this parallel run (total 300)...
Building estimator 4 of 300 for this parallel run (total 300)...
Building estimator 5 of 300 for this parallel run (total 300)...
Building estimator 6 of 300 for this parallel run (total 300)...
Building estimator 7 of 300 for this parallel run (total 300)...
Building estimator 8 of 300 for this parallel run (total 300)...
Building estimator 9 of 300 for this parallel run (total 300)...
Building estimator 10 of 300 for this parallel run (total 300)...
Building estimator 11 of 300 for this parallel run (total 300)...
Building estimator 12 of 300 for this parallel run (total 300)...
Building estimator 13 of 300 for this parallel run (total 300)...
Building estimator 14 of 300 for this parallel run (total 300)...
Buil

Building estimator 124 of 300 for this parallel run (total 300)...
Building estimator 125 of 300 for this parallel run (total 300)...
Building estimator 126 of 300 for this parallel run (total 300)...
Building estimator 127 of 300 for this parallel run (total 300)...
Building estimator 128 of 300 for this parallel run (total 300)...
Building estimator 129 of 300 for this parallel run (total 300)...
Building estimator 130 of 300 for this parallel run (total 300)...
Building estimator 131 of 300 for this parallel run (total 300)...
Building estimator 132 of 300 for this parallel run (total 300)...
Building estimator 133 of 300 for this parallel run (total 300)...
Building estimator 134 of 300 for this parallel run (total 300)...
Building estimator 135 of 300 for this parallel run (total 300)...
Building estimator 136 of 300 for this parallel run (total 300)...
Building estimator 137 of 300 for this parallel run (total 300)...
Building estimator 138 of 300 for this parallel run (total 300